# Preliminary Setup and Constants

! Run the following code before anything else in the notebook. These are project wide imports and constants

In [ ]:
import tensorflow as tf
import os

LOCAL_PATH = os.path.dirname(os.path.abspath(__file__))
WORKSPACE_PATH = LOCAL_PATH + 'Tensorflow/workspace'
SCRIPTS_PATH = LOCAL_PATH + 'Tensorflow/scripts'
APIMODEL_PATH = LOCAL_PATH + 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH + '/annotations'
IMAGE_PATH = WORKSPACE_PATH + '/images'
MODEL_PATH = WORKSPACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + '/pre-trained-models'
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/'

# Step 1: Labelling

Creating the label map file

In [ ]:
labels = [{'name':'0', 'id':0},{'name':'1', 'id':1},{'name':'2', 'id':2},{'name':'3', 'id':3},{'name':'4', 'id':4}, {'name':'5', 'id':5},{'name':'6', 'id':6},{'name':'7', 'id':7},{'name':'8', 'id':8},{'name':'9', 'id':9}]
number_imgs = 204

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')


# Step 2: Tensoflow Records Generation

In [ ]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

# Step 3: Model Configuration

### A. Downloading and organizing the SSD model

Do the following steps to clone the tensoflow models library to the tensorflow folder:
1. Navigate into the Tensorflow folder in the project
2. Run the following command in the terminal: git clone https://github.com/tensorflow/models

Next, download the deep learning single shot detector model (SSD) from tensorflow to the pretrained model path using the following code (these are terminal level commands executed by python):

### B. Copying the pipeline.config file

In [ ]:
wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

Make a directory in `Tensorflow\workspace\models` called `\my_ssd_mobnet` for where we will place our custom model into.

Next we must copy the pretrained model's pipeline.config file from folder we downloaded above (called `ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz`) located in the Tensorflow\workspace\pre-trained-models folder and into our custom model's folder: `Tensorflow\workspace\models\my_ssd_mobnet`

### C. Updating the copied pipeline.config file

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config #Runs in the terminal


In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config) 

pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [ ]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

# Step 4: Model Training

In [ ]:
!"""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME

# Step 5: Real Time Detection

# Step 6: Real Time Code Generation